# Initiation au Parsing
## Partie 1

# Installer le projet
/!\ Faites ceci avant de commencer votre exploration de ce notebook, puisque le notebook a besoin d'avoir certaines dépendances d'installées


Dans votre terminal, lancez 
```
bash install.sh
```

Ce script :
- clone le projet github `stackexchange_dataset`
- crée un environnement virtuel python
- installe les dépendences nécéssaires

## Récupérer les données
Dans votre terminal, lancez 
```
bash download_data
```
Ce script va telecharger les forums stackexchange demandés. Vous pouvez donc choisir ceux de votre choix. Regardez les commentaires du script pour voir comme modifier la liste de forum à récupérer.

## Pré-code
On désactive l'affichage des FutureWarning (librairies, méthodes ou arguments qui vont être deprecated dans le futur)

In [14]:
import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



## Préparer les textes
Il nous faut dézipper les fichiers avant de pouvoir les utiliser

In [15]:
# On définit d'abord notre liste de corpus
CORPUS_NAMES =  'politics,french,mythology,woodworking,hsm,health,portuguese'.split(',')

In [16]:
# On import les librairies nécessaires pour cette partie
import shutil, os

# Puis on dézippe les fichiers
os.makedirs('texts', exist_ok=True)

for corpus_name in CORPUS_NAMES:
    print("Dezippage de :", corpus_name)
    shutil.unpack_archive('./out/'+corpus_name+'.stackexchange.zip', './texts/'+corpus_name)
print("Terminé")

Dezippage de : politics
Dezippage de : french
Dezippage de : mythology
Dezippage de : woodworking
Dezippage de : hsm
Dezippage de : health
Dezippage de : portuguese
Terminé


### Essayont d'analyser un peu nos données brutes


In [17]:
corpora = {}
corpus_info = []
MAX_FILES = 5000
for corpus_name in CORPUS_NAMES:
    total_text = ''
    print(corpus_name)
    files_names = os.listdir('texts/'+corpus_name+'/')
    for file in files_names[:MAX_FILES]:
        total_text += open('texts/'+corpus_name+'/'+file).read()
    corpora[corpus_name] = total_text
    corpus_info+=[{'corpus':corpus_name, 'nrFiles':len(files_names), 'nrChars':len(total_text)}]
print(corpus_info)

politics


On y voit pas grand chose, essayons d'utiliser pandas, une librairie de "tableur automatisés" pour comprendre notre dataset

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(corpus_info, index=['corpus'])
df

In [ ]:
df['text'] = [corpora[c] for c in corpora]
df

Visualisons les premières lignes de chaque forum 

In [ ]:
for c, t in corpora.items():
    print('___',c,t[:50])

## Faisons quelques graphes de nos corpus

In [ ]:
import seaborn as sns
sns.barplot(x = df.index, y = 'nrFiles', data = df, palette='husl')

In [ ]:
sns.barplot(x = df.index, y = 'nrChars', data = df, palette='husl')

In [ ]:
df['charsPerFile']=df['nrChars']/df['nrFiles']
sns.barplot(x = df.index, y = 'charsPerFile', data = df, palette='husl')

In [ ]:
import collections
typeChars = []
for c, t in corpora.items():
    counter = collections.Counter(t)
    typeChars +=[len(counter)]
    print(c,counter)
df['typeChars']=typeChars

## Les Mots et Tokens

### tokenizing

In [ ]:
# Comment pouvons nous tokeniser la phrase ?
text = "Let's all together defeat last year's problem, SARS-CoV-2, in 2022!"
splitted_text = ...

In [ ]:
# Avec re, on peut utiliser la fonction split pour tokeniser
import re
nochar_regex = re.compile('\W+') # On définit une expression régulière pour les caractères non-alphanumériques
'|'.join(nochar_regex.split(text)),len(nochar_regex.split(text))

NameError: name 'text' is not defined

In [ ]:
charorhyphen_regex = re.compile(r'[\w-]+') # On définit une expression régulière pour les caractères alphanumériques et le tiret
'|'.join(charorhyphen_regex.findall(text)),len(charorhyphen_regex.findall(text))

### NLTK : word_tokenize
On va utiliser la fonction de tokenization de nltk, qui marche correctement pour l'anglais. Si l'on souhaite tokeniser d'autres langues, on peut utiliser les sous modules nltk concernés ou bien ceux de spacy.
- Pour le chinois, jieba est une bonne solution.
- Pour le japonais, mecab.
- Pour le thai, pythai-nlp.

- Pour les autres langues compliquées sur la tokenisation, il faut faire ses propres recherches avant de se lancer



In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

'|'.join(word_tokenize(text)),len(word_tokenize(text))
# Donc pour tokeniser en français : word_tokenize(text, language='french')

### Par contre c'est plutôt lent :

In [ ]:
from tqdm import tqdm
for c, t in tqdm(corpora.items()):
    toks = word_tokenize(t[:1000000])


Soyez patient pour cette ligne. Ça a pris 44s sur mon pc

In [ ]:
df['tokens'] = df['text'].map(word_tokenize)
df['nrTokens'] = df['tokens'].map(len)
df['nrTypes'] = df['tokens'].map(set).map(len)

df

In [ ]:
df['avg_wordsize'] = df['nrChars']/df['nrTokens']
sns.barplot(x = df.index, y = 'avg_wordsize', data = df, palette='husl')

In [ ]:
# On peut utiliser la fonction Counter de la librairie collections pour compter les tokens
counter = collections.Counter(df['tokens']['mythology'])
freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['mythology'])

# Quel est le mot le plus fréquent dans le corpus mythology ?
freq_df.sort_values('mythology',  inplace=True, ascending=False)
freq_df.head(22)

In [ ]:
wordcounter = collections.Counter(df['tokens']['mythology'])


#### 🚧 todo:
Si vous voulez faire un peu de fouille de données, vous pouvez répondre aux questions ci-dessous :

In [ ]:
print('average length in the dictionary (on types):',sum(...
print('average length in the text (on tokens): ',sum(...
print('the longest words: ', sorted(...

In [ ]:
# remove solution:
print('average length in the dictionary (on types):',sum([len(t) for t in wordcounter])/len(wordcounter))
print('average length in the text (on tokens): ',sum([len(t)*f for t,f in wordcounter.items()])/ sum(wordcounter.values()))
print('the longest words: ', sorted(wordcounter, key=lambda x:len(x), reverse=True)[:10])

In [ ]:
lenfreq = {} # will contain word length to frequency in the dictionary 
for t,f in wordcounter.items():
    lenfreq[len(t)] = lenfreq.get(len(t),0)+1 # if replacing +1 by + f, we have the length / frequency relation in the texts 
print(lenfreq)

In [ ]:
freq_df = pd.DataFrame.from_dict(lenfreq, 
                                 orient='index', 
                                 columns=['frequency']).sort_values(by='frequency', ascending=False)
                                #
# freq_df.sort_index(inplace=True)
freq_df.head()

In [ ]:
freq_df["frequency"].plot(kind='bar', title='check the long tail!')

In [ ]:
freq_df.head(15).plot(kind='bar', title='without the long tail')

### different corpus, same graph

In [ ]:
wordcounter = collections.Counter(df['tokens']['woodworking'])
lenfreq = {} 
for t,f in wordcounter.items():
    lenfreq[len(t)] = lenfreq.get(len(t),0)+1 
print(lenfreq)
freq_df = pd.DataFrame.from_dict(lenfreq, orient='index', columns=['frequency'])
freq_df.sort_index(inplace=True)
display(freq_df.head())
freq_df.head(15).plot(kind='bar', title='check the long tail!')

-----------

## Faisons nos vecteurs de mots
On utilise Gensim, une librairie de NLP qui est particulierement facile d'accès pour les plongements vectoriels

In [ ]:
from gensim.models import Word2Vec
from typing import Dict

## Les hyperparamètres du modèle
Les "hyperparamètres", en ML, sont les paramètres d'entrainement d'un modèle que l'utilisateur peut modifier pour influer sur la qualité du modèle. Ce sont ces hyperparamètres que l'on va chercher à optimiser en entrainant plusieurs modèles et en selectionnant celui qui performe le mieux sur les fonctions de couts et d'évaluation

Dans notre cas, vous avez plusieurs hyperparamètres possibles :
- `vector_size` : taille des vecteurs d'embedding 
- `epochs` : le nombre de fois que le modèle s'entrainte sur la totalité des données
- `window` : la taille de la fenêtre de contexte autour du mot à prédire
- `min_count` : nombre d'occurence mini pour un mot pour être dans le vocab


In [ ]:
embeddings_models: Dict[str, Word2Vec] = {} # On ajoute du typage statique pour plus de clarté
for c in df.index:
    print(c)
    embeddings_models[c] = Word2Vec([df['tokens'][c]], min_count=5, epochs=100, vector_size=100, window=5)

In [ ]:
similarity_scores = {}
def get_similarity_scores(embeddings_models, word1, word2):
    for model_name in embeddings_models.keys():
        model = embeddings_models[model_name]
        try :
            score = model.wv.similarity(word1, word2)
        except KeyError:
            score = "NOT_FOUND"
        similarity_scores[model_name] = score
    return similarity_scores

def print_similarity_scores(embeddings_models, word1, word2):
    print(f"Computing sim({word1}, {word2})...")
    scores = get_similarity_scores(embeddings_models, word1, word2)
    for c, s in scores.items():
        print(f"sim({word1}, {word2}) in {c}: {s}")
    print()
    

print_similarity_scores(embeddings_models, "sun", "moon")
print_similarity_scores(embeddings_models, "apple", "grape")
print_similarity_scores(embeddings_models, "apple", "orange")
print_similarity_scores(embeddings_models, "nose", "mouth")



In [ ]:
# Si vous voulez la matrice d'embeddings pour un corpus donné :
embeddings = model.wv.get_normed_vectors()

In [ ]:
embeddings_models["mythology"].wv.most_similar('sun', topn=10)

Pouvez vous savoir pourquoi les résultats sont mauvais ? Essayez de regarder à nouveau la distribution des mots les plus fréquents

In [ ]:
import gensim.downloader

# Show all available models in gensim-data

print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
glove_twitter_25 = gensim.downloader.load('glove-twitter-25')

In [ ]:
# Quel va être le mot le plus proche de "sun" ?
glove_twitter_25.most_similar('sun', topn=10)

In [ ]:
# Et quel va être le mot le plus proche de "apple" ?
...

In [ ]:
# Mais est-ce que "apple" est quand même proche des autres fruits ?
glove_twitter_25.similarity('apple', 'orange')

Maintenant que l'on a nos vecteurs de mots, on va pouvoir commencer à les utiliser en entrée de nos parseurs. 
On se garde ça pour la semaine prochaine 🙌